In [1]:
import pandas as pd 

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data;

In [3]:
data['Tenure'].max()

np.int64(10)

In [4]:
data.isnull().sum();

In [5]:
data.drop(columns = {'RowNumber','CustomerId','Surname'},inplace = True)

In [6]:
print(data['Gender'].unique())
print(data['Geography'].unique())

['Female' 'Male']
['France' 'Spain' 'Germany']


In [7]:
# Gender column -> we can solve by LabelEncoder 
# Geography column-> we can solve by OneHotEncoder

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])

onehot_encoder = OneHotEncoder(sparse_output=False)
encoded = onehot_encoder.fit_transform(data[['Geography']])
encoded = pd.DataFrame(encoded,columns = onehot_encoder.get_feature_names_out(['Geography']))
encoded.rename(columns = {'Geography_France':'France', 'Geography_Germany':'Germany', 'Geography_Spain':'Spain'},inplace = True)
data = pd.concat([data,encoded],axis = 1)

In [8]:
import pickle
with open('gender.pkl','wb') as file:
    pickle.dump(label_encoder,file)
with open('geography.pkl','wb') as file:
    pickle.dump(onehot_encoder,file)

In [9]:
feature = data.drop(columns = ['Exited','Geography'],axis = 1)
target = data['Exited']

In [10]:
# Splitting of the data 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(feature,target,train_size=0.8,random_state=42)

In [11]:
# We will use the scalar for giving the value equal weightage
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [13]:
#Steps to go in ANN Classification 
# 1 - Feature Engineering 
# 2 - ANN Model Creation/ Sequential Diagram (Hidden layer neuron + activation function + input)
# 3 - Compile the Model/backpropogation (optimizer + loss function + metrics)
# 4 - Callbacks to reduce the duration of the model running
# 5 - test the model
# 6 - logs visualization through tensorboard

In [14]:
## ANN ka skeleton tyyar hua h -> kitne neuron + activation + kitne input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(64,activation = 'relu',input_shape = (X_train.shape[1],)),
    Dense(34,activation = 'relu'),
    Dense(1,activation = 'sigmoid')
])

c:\Git Repository\ANNClassifier\ANNenv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
## Back propagation ki tyyari -> loss function konsa + metrics me kya + optimizer konsa rhega
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

opt = Adam(learning_rate = 0.01)
loss = BinaryCrossentropy()

model.compile(loss = loss, optimizer = opt,metrics = ['accuracy'])

In [16]:
## model ke training Params ko log me collect krte h 
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard

log_dir = "log/fit/" + datetime.now().strftime("%Y%m%D-%H%M%S")
log_visual = TensorBoard(log_dir = log_dir, histogram_freq =1 )


In [17]:
#Model jha pe convergance pe aa jaye wha se hi model training exit kr dega aur best params store kr lega
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(patience = 10,restore_best_weights = True)

In [18]:
## Ab model ko fit krte h apne data ke saath 
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs = 50,
          callbacks = [log_visual,early_stop])

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6747 - loss: 312.8224 - val_accuracy: 0.8035 - val_loss: 95.9806
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6721 - loss: 14.6998 - val_accuracy: 0.7950 - val_loss: 1.2879
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7462 - loss: 1.6024 - val_accuracy: 0.8040 - val_loss: 0.5397
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7880 - loss: 0.5348 - val_accuracy: 0.8035 - val_loss: 0.4944
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7912 - loss: 0.5161 - val_accuracy: 0.8010 - val_loss: 0.4953
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7925 - loss: 0.5092 - val_accuracy: 0.8035 - val_loss: 0.4988
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7930 - loss: 0.5053 - val_accuracy: 0.8035 - val_loss: 0.4941
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7943 - loss: 0.5046 - val_accuracy

In [19]:
#saving the whole model 
model.save('ANNModel.h5')

In [20]:
##To visualize the logs 
%load_ext tensorboard

In [21]:
%tensorboard --logdir log/fit/20251010

Reusing TensorBoard on port 6006 (pid 5248), started 1 day, 2:59:08 ago. (Use '!kill 5248' to kill it.)